In [1]:
import pandas as pd
import re
#import sqlite3
import datetime
import os
import numpy as np
from math import ceil 
#from openpyxl import Workbook, load_workbook

In [54]:

for fi in os.listdir(fp):
    if re.search('^\d+', fi):
        content = pd.read_csv(f'{fp}{fi}', sep='\t') 
        fi = fi.split('_')[2].split('.')[0]
        with open(f"{fp}gilson_set_file_{fi}.txt", 'w') as f:
            f.write('Sample_Name\tTube_Barcode\tBrooks_Barcode\n')

            for i,r in content.iterrows():
                 if i > 0 and re.search('^[L|M|C]', r['SampleName[True,108,2]']):
                        #print(r['SampleName[True,108,2]'],'\t',r['NOTES_STRING[False,68,9]'],'\t', r['SampleDescription[False,120,4]'])
                        f.write(f"{r['SampleName[True,108,2]']}\t{r['SampleDescription[False,120,4]']}\t{r['NOTES_STRING[False,68,9]']}\n")

In [10]:
!ls $fp 

1520030001_secStg_127.tsl
1520030002_secStg_134.tsl
1520030003_secStg_138.tsl
1520060001_secStg_15200600010.tsl
1520060002_secStg_15200600020.tsl
1556_test.tsl
15_test.tsl
20190603_2_M21523_5_(4)_2020-07-26_09-40-30-PM_at_03-32-39-PM_(2).csv
20190603_2_M21535_5_(4)_2020-07-26_09-40-30-PM_at_03-32-39-PM_(2).csv
20190603_2_M21551_5_(4)_2020-07-26_08-40-30-PM_at_03-32-39-PM_(2).csv
20190603_2_M24173_2_(4)_2020-07-26_08-40-30-PM_at_03-32-39-PM_(2).csv
20190603_3_M23127_6_(4)_2020-07-26_08-40-30-PM_at_03-32-39-PM_(2).csv
20190603_3_M23131_6_(4)_2020-07-26_08-40-30-PM_at_03-32-39-PM_(2).csv
20190603_5_L88699_3_(3)_2019-06-04_09-05-59-AM_at_09-41-53-AM_(2).csv
20190605_3_L40885_3_(2)_2020-07-26_10-35-12-PM_at_10-55-18-AM_(2).csv
20200726_6_L7547_3_(3)_2020-07-26_09-30-20-PM_at_10-46-24-PM_(2).csv
20200726_6_L88759_2_(3)_2020-07-26_09-30-20-PM_at_10-46-24-PM_(2).csv
gilson_set_file_127.txt
gilson_set_file_134.txt
gilson_set_file_138.txt


In [4]:
rack1 = '1520030001'
rack2 = '1520030002'
rack3 = '1520060001'
rack4 = '1520060002'

In [24]:
fp = '/users/trinhsk/documents/GitRepos/gilson_webapp/test_files/'

#conn = sqlite3.connect(f'{fp}gilsontsl.db')
#c = conn.cursor()

In [3]:

def srchmap_tslfile_dictdf(list_of_rack_nbrs, tslfile_dir):
    '''search a given directory for tsl files that match rack number and store
    pd df into dictionary'''
    dct_df = {}
    for rk in list_of_rack_nbrs:
        for fi in os.listdir(tslfile_dir):
            if re.search(fr'^{rk}_', fi):
                dct_df[rk] = pd.read_csv(
                    os.path.join(tslfile_dir, fi), sep='\t')
                if rk == list_of_rack_nbrs[0]:
                    continue
                else:
                    break
    return dct_df



In [6]:
dct_df = srchmap_tslfile_dictdf([rack1, rack2], fp)

In [7]:
def find_end_idx_sw(df):
    '''find the end index row and sample well number for the first rack'''
    for i in range(df.shape[0]-1,0,-1):
        sw_value = df.loc[i, '#Sample Well[True,109,10]']
        if sw_value != 0:
            return i, sw_value

In [8]:
def incr_plt_loc(plate_loc, incr):
    suffix_nbr = plate_loc[-3:]
    curr_val = int(re.search(r'\d{2}S', plate_loc).group(0)[:2]) + incr 
    if curr_val < 10:
        curr_val = f'0{curr_val}'
    return f'P{curr_val}{suffix_nbr}'

In [9]:
def get_end_plt_idx(df, end_idx):
    plate_loc = df.loc[end_idx,'#Plate_Sample[True,115,13]']
    return int(re.search(r'\d{2}S', plate_loc).group(0)[:2])

In [107]:
conditions = df['#Plate_Sample[True,115,13]'].apply(lambda x : x[:3]).unique()
conditions

array(['P01', 'P02', 'P03', 'P04', 'P05', 'P06'], dtype=object)

In [110]:
conditions = [df['#Plate_Sample[True,115,13]'].str.contains(pl_prx) for pl_prx in conditions]

In [115]:
np.select(conditions, choices, default=np.nan).shape

(24,)

In [113]:
choices = colour*int(unq_prefx_plt.shape[0]/2)

In [118]:
def base_combine_worklists(df1, df2):
    '''combine two worklists using pandas''' 
    sw_buffer = 0
    # remove rows that don't have sample well number
    df1 = df1[~df1['#Plate_Sample[True,115,13]'].isna()]
    df2 = df2[~df2['#Plate_Sample[True,115,13]'].isna()]
    df1.reset_index(inplace=True, drop=True)
    df2.reset_index(inplace=True, drop=True)

    row_cnt = df1.shape[0]
    if row_cnt % 4 != 0:
        # add to the incr counters to make sample wells in the next iteration
        sw_buffer = 4 - (row_cnt % 4)
        
    # new sample well starting number for second rack (df2)
    end_idx, start_sw = find_end_idx_sw(df1)
    print(end_idx)
    start_sw += sw_buffer
    incr = get_end_plt_idx(df1, end_idx)
    print(f'incr: {incr}')
    print(f'sw_buffer: {sw_buffer}')
    # re-assign sample well as a increment of last sample well value from first rack (df1) 
    df2.loc[:, '#Sample Well[True,109,10]'] = df2['#Sample Well[True,109,10]'].map(lambda x: x + start_sw)
    df2.loc[:, '#Plate_Sample[True,115,13]'] = df2['#Plate_Sample[True,115,13]'].map(lambda x: incr_plt_loc(x, incr))
    df = pd.concat([df1, df2])
    #print(df.shape)
   
    colour = ['#f5f5f5','#e0dede']
    # make conditions for assigning colours; first get unique plate location prefixes i.e. 'P01'
    unq_prefx_plt = df['#Plate_Sample[True,115,13]'].apply(lambda x : x[:3]).unique()
    # second, use a regex contains string to compare and get list of list of booleans that show matches
    conditions = [df['#Plate_Sample[True,115,13]'].str.contains(pl_prx) for pl_prx in unq_prefx_plt]
    # colour assignment hex-colour (2 options) that will be selected based on the position of the condition 
    choices = colour*int(unq_prefx_plt.shape[0]/2)
    df['colour'] = np.select(conditions, choices, default=np.nan) #np.tile(np.repeat(colour, 4), int(df.shape[0]/4/2))

    return df

In [11]:
def insert_row_(row_number, df, row_value): 
    df_result = pd.DataFrame() 
    if isinstance(row_number, list):
        for rn in row_number:
            if not df_result.empty:
                df = df_result
            df1 = df[0:rn] 
            df2 = df[rn:] 
            if row_value.shape[0] > 1:
                for i in range(row_value.shape[0]):
                    df1.loc[rn+i]=row_value[i]
            else:
                df1.loc[rn]=row_value
            df_result = pd.concat([df1, df2]) 
            df_result.index = [*range(df_result.shape[0])] 
    else:
        # if just one row number to insert at
        df1 = df[0:row_number] 
        df2 = df[row_number:] 
        if row_value.shape[0] > 1:
            # if multiple rows to insert
            for i in range(row_value.shape[0]):
                df1.loc[row_number+i]=row_value[i]
        else:
            df1.loc[row_number]=row_value
        df_result = pd.concat([df1, df2]) 
        df_result.index = [*range(df_result.shape[0])] 
   
    return df_result 

In [69]:
def main_combine_worklists(df1, df2):
    startup_rows = df1.iloc[:3]
    startup_rows['colour'] = np.nan
    shutdown_rows = df1.iloc[-3:]
    shutdown_rows['colour'] = np.nan
    # strip flush/std/startup/shutdown for easier manipulation
    base_df = base_combine_worklists(df1, df2)
    # list of row index nbrs that will have the flush/6-cmpd std; multiple of 10
    list_insert_idx = [*range(10, ceil(base_df.shape[0] /10)*10, 10)]
    # add 2 every iteration since that is the nbr of rows being added (shift over)
    list_insert_idx = [i+2*e if e!=0 else i for e,i in enumerate(list_insert_idx)]
    # add colour column
    flush_std_rows = df1.iloc[1:3]
    flush_std_rows['colour'] = np.nan
    # insert flush and 6-cmpd std; index the 1st/3rd rows since 0th is startup
    main_df = insert_row_(list_insert_idx, base_df, flush_std_rows.values)
    # add first 3 rows add last three rows
    main_df = pd.concat([startup_rows, main_df, shutdown_rows])
    main_df.reset_index(drop=True, inplace=True)
    return main_df 

In [119]:
dfc = main_combine_worklists(dct_df['1520030001'], dct_df['1520030002'])

/Users/trinhsk/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/trinhsk/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


23
incr: 6
sw_buffer: 0


/Users/trinhsk/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/trinhsk/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [120]:
dfc.head()

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]",colour
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,0,FB02646205,NaN,YES,UNKNOWN,...,127,1,P01S01,#f5f5f5
4,RUN,4ML_10mm_2ND STAGE,M21551_5,0,FB02646225,NaN,YES,UNKNOWN,...,127/12,2,P01S02,#f5f5f5


In [72]:
dfc.columns

Index(['SkipPause[True,18,0]', 'MethodName[True,239,1]',
       'SampleName[True,108,2]', 'SampleAmount[False,78,3]',
       'SampleDescription[False,120,4]', 'SampleId[False,90,5]',
       'InjectionYesNo[False,72,6]', 'PeakInformation[False,104,7]',
       'SampleInjection[False,138,8]', 'NOTES_STRING[False,68,9]',
       '#Sample Well[True,109,10]', '#Plate_Sample[True,115,13]', 'colour'],
      dtype='object')

In [121]:
dfc.iloc[:, [0,1,2,4,9,10,11,12]]

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleDescription[False,120,4]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]",colour
0,RUN,4 ML STARTUP,Sample,NaN,NaN,0,NaN,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,NaN,NaN,0,NaN,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,NaN,NaN,0,NaN,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,FB02646205,127,1,P01S01,#f5f5f5
4,RUN,4ML_10mm_2ND STAGE,M21551_5,FB02646225,127/12,2,P01S02,#f5f5f5
5,RUN,4ML_10mm_2ND STAGE,M21523_5,FB02646185,127/12,3,P01S03,#f5f5f5
6,RUN,4ML_10mm_2ND STAGE,M24173_2,FB03658814,127/12,4,P01S04,#f5f5f5
7,RUN,4ML_10mm_2ND STAGE,M23131_6,FB02699823,127/12,5,P02S01,#e0dede
8,RUN,4ML_10mm_2ND STAGE,M23127_6,FB02699828,127,6,P02S02,#e0dede
9,RUN,4ML_10mm_2ND STAGE,L7547_3,FB02928213,127/12,7,P02S03,#e0dede


In [126]:
['none' if pd.isna(x) else x for x in dfc.colour]

['none',
 'none',
 'none',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 '#f5f5f5',
 '#f5f5f5',
 'none',
 'none',
 '#f5f5f5',
 '#f5f5f5',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 'none',
 'none',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#e0dede',
 '#e0dede',
 'none',
 'none',
 '#e0dede',
 '#e0dede',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 'none',
 'none',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#f5f5f5',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 '#e0dede',
 'none',
 'none',
 'none']

In [67]:
conn.close()

In [6]:
sqlite_create_table_query = '''CREATE TABLE GILSON_SET_TABLE 
                            (RUN_DATE TIMESTAMP,
                            ID TEXT NOT NULL, 
                            SAMPLE_NUMBER INTEGER NOT NULL,
                            SAMPLE_NAME TEXT NOT NULL, 
                            TUBE_BARCODE TEXT PRIMARY KEY, 
                            BROOKS_BARCODE TEXT NOT NULL);'''

sqlite_delete_table_query = '''DROP TABLE GILSON_SET_TABLE;'''

#c.execute(sqlite_delete_table_query)
c.execute(sqlite_create_table_query)

In [11]:
for fi in os.listdir(fp):
    if re.search('^\d+', fi):
        split_fi_name = fi.split('_')
        suffix = split_fi_name[2].split('.')[0]
        input_file = f"{fp}gilson_set_file_{suffix}.txt"
        fi_content = pd.read_csv(input_file, sep='\t')
        num_rows = len(fi_content.index)
        for i,r in fi_content.iterrows():
            numbr = i % num_rows
            ls_tp_content = [(datetime.datetime.now(), split_fi_name[0], numbr, r['Sample_Name'], r['Tube_Barcode'], str(int(r['Brooks_Barcode'])))]
            c.executemany("INSERT INTO GILSON_SET_TABLE VALUES (?,?,?,?,?,?)", ls_tp_content)
            
        conn.commit()

In [50]:
wb_load = load_workbook(f"{fp}excel_gilson_script_test.xlsx")
print(wb_load.sheetnames)

['Sheet1', 'Set1', 'Set2']


In [51]:
wset1 = wb_load['Set1']
wset2 = wb_load['Set2']

In [52]:
ids = c.execute('''SELECT DISTINCT ID FROM GILSON_SET_TABLE''').fetchall()
for rw in ids:
    print(rw)

('1520030001',)
('1520030002',)
('1520030003',)


In [53]:
project_num = '1520030001'

for idt in ids:
    if project_num in idt:
        row_data = c.execute(f'''SELECT * FROM GILSON_SET_TABLE WHERE ID = {idt[0]}''').fetchall()

#for i,row in enumerate(row_data):
#    print(row)

In [54]:
header = ['RUN_DATE', 'ID', 'SAMPLE_NUMBER', 'SAMPLE_NAME', 'TUBE_BARCODE', 'BROOKS_BARCODE']
for i in range(1,7):
    #print(chr(64+i))
    wset1[f"{chr(64+i)}1"] = header[i-1]

In [55]:
rcnt=0
ccnt=0
for row in range(2, len(row_data)+1):
    for col in range(1,7):
        _ = wset1.cell(column=col, row=row, value=f"{row_data[rcnt][ccnt%6]}")
        #print(row_data[rcnt][ccnt%6])
        #print(f"{ccnt%6} {rcnt}")
        ccnt+=1
    rcnt+=1

In [19]:
#for i in range(3,len(row_data)):
#    wset1.append(row_data[i])
    #print(row_data[i])

In [67]:
#wset1['A10'].value

In [64]:
def update_set_data(project_id, set_num, excel_file = 'excel_gilson_script_test'):
    wb_load = load_workbook(f"{fp}{excel_file}.xlsx")
    wset = wb_load[f"Set{set_num}"]
    row_data = c.execute(f'''SELECT * FROM GILSON_SET_TABLE WHERE ID = {project_id}''').fetchall()
    brooks_bc = c.execute(f'''SELECT DISTINCT BROOKS_BARCODE FROM GILSON_SET_TABLE WHERE ID = {project_id}''').fetchone()
    #clear content of sheet
    for row in wset['A1:H24']:
        for cell in row:
            cell.value = None
    #add header to sheet
    for i in range(1,7):
        wset[f"{chr(64+i)}1"] = header[i-1]
    #populate the cells
    rcnt=0
    ccnt=0
    for row in range(2, len(row_data)+1):
        for col in range(1,7):
            _ = wset.cell(column=col, row=row, value=f"{row_data[rcnt][ccnt%6]}")
            ccnt+=1
        rcnt+=1
    #overwrite the excel file
    wb_load.save(f'{fp}{excel_file}.xlsx')
    print('completed update')
    
def save_tsl_file(pid_1, pid_2, brooks_bc1, brooks_bc2):
    #save new tab-delimited file to directory
    parent_dir = '/Volumes/npsg/tecan/SourceData/SecondStage/Completed Projects Injections/'
    pid = pid_1[:8]
    pid_suffix1 = pid_1[8:]
    pid_suffix2 = pid_2[8:]
    for fi in os.listdir(parent_dir):
        if fi.startswith(pid) and os.path.isdir(os.path.join(parent_dir,fi)):
            file_path = f'{parent_dir}/3_SecondStage Sample List/{pid}_{pid_suffix1}_{pid_suffix2}_{brooks_bc1}_{brooks_bc2}.tsl'
    with open(file_name, 'w') as fw:
        wf.write('\n')

In [65]:
update_set_data('1520030002',1)

134


In [2]:
data = {
    "BARCODE": "FB02646225",
    "BROOKS_BARCODE": "127",
    "FINISH_DATE": "Mon, 03 Aug 2020 15:17:01 GMT",
    "GILSON_NUMBER": "FR-N-S081093",
    "METHOD_NAME": "4ML_10mm_2ND STAGE",
    "PLATE_ID": "11",
    "PLATE_POSITION": "P01S02",
    "PROJECT_ID": "15200300",
    "SAMPLE_NAME": "M21551_5",
    "SAMPLE_WELL": "2",
    "SEQ_NUM": 3,
    "TSL_FILEPATH": "15200300_001_002_comb.tsl",
    "UVDATA": 4}

In [3]:
for k,v in data.items():
    print(k,v)

BARCODE FB02646225
BROOKS_BARCODE 127
FINISH_DATE Mon, 03 Aug 2020 15:17:01 GMT
GILSON_NUMBER FR-N-S081093
METHOD_NAME 4ML_10mm_2ND STAGE
PLATE_ID 11
PLATE_POSITION P01S02
PROJECT_ID 15200300
SAMPLE_NAME M21551_5
SAMPLE_WELL 2
SEQ_NUM 3
TSL_FILEPATH 15200300_001_002_comb.tsl
UVDATA 4


In [7]:
def sort_keys(dct):
    key_name_list = [
    'SEQ_NUM',
    'FINISH_DATE',
    'PROJECT_ID',
    'METHOD_NAME',
    'SAMPLE_NAME',
    'BARCODE',
    'PLATE_ID',
    'BROOKS_BARCODE',
    'SAMPLE_WELL',
    'PLATE_POSITION',
    'GILSON_NUMBER',
    'TSL_FILEPATH',
    'UVDATA',
    ]

    dct_ = {k: data[k] for k in key_name_list}
    return dct_


In [8]:
sort_keys(data)

{'SEQ_NUM': 3,
 'FINISH_DATE': 'Mon, 03 Aug 2020 15:17:01 GMT',
 'PROJECT_ID': '15200300',
 'METHOD_NAME': '4ML_10mm_2ND STAGE',
 'SAMPLE_NAME': 'M21551_5',
 'BARCODE': 'FB02646225',
 'PLATE_ID': '11',
 'BROOKS_BARCODE': '127',
 'SAMPLE_WELL': '2',
 'PLATE_POSITION': 'P01S02',
 'GILSON_NUMBER': 'FR-N-S081093',
 'TSL_FILEPATH': '15200300_001_002_comb.tsl',
 'UVDATA': 4}

In [5]:
dct_

{'SEQ_NUM': 3,
 'FINISH_DATE': 'Mon, 03 Aug 2020 15:17:01 GMT',
 'PROJECT_ID': '15200300',
 'METHOD_NAME': '4ML_10mm_2ND STAGE',
 'SAMPLE_NAME': 'M21551_5',
 'BARCODE': 'FB02646225',
 'PLATE_ID': '11',
 'BROOKS_BARCODE': '127',
 'SAMPLE_WELL': '2',
 'PLATE_POSITION': 'P01S02',
 'GILSON_NUMBER': 'FR-N-S081093',
 'TSL_FILEPATH': '15200300_001_002_comb.tsl',
 'UVDATA': 4}

In [4]:
lst_rows =[['7/30/2020 2:15:20 PM', '4', '4ML_10mm_2ND_STAGE', '4', 'None', '1', '1', 'P01S01'],
    ['7/30/2020 2:55:20 PM', '5', '4ML_10mm_2ND_STAGE', '5', 'None', '2', '2', 'P01S02'],
    ['7/30/2020 3:45:50 PM', '6', '4ML_10mm_2ND_STAGE', '6', 'None', '3', '3', 'P01S03'],
    ['7/30/2020 4:52:50 PM', '7', '4ML_10mm_2ND_STAGE', '7', 'None', '4', '4', 'P01S04'],
    ['7/30/2020 5:33:50 PM', '8', '4ML_10mm_2ND_STAGE', '8', 'None', '5', '5', 'P02S01'],
    ['7/30/2020 6:41:50 PM', '9', '4ML_10mm_2ND_STAGE', '9', 'None', '6', '6', 'P02S02'],
    ['7/30/2020 7:12:50 PM', '10', '4ML_10mm_2ND_STAGE', '10', 'None', '7', '7', 'P02S03'],
    ['7/30/2020 8:09:50 PM', '11', '4ML_10mm_2ND_STAGE', '11', 'None', '8', '8', 'P02S04'],
    ['7/30/2020 8:16:40 PM', '12', '4ML_10mm_2ND_STAGE', '12', 'None', '9', '9', 'P03S01'],
    ['7/30/2020 9:09:18 PM', '13', '4ML_10mm_2ND_STAGE', '13', 'None', '10', '10', 'P03S02'],
    ['7/30/2020 11:02:18 PM', '14', '4ML_10mm_2ND_STAGE', '14', 'None', '13', '13', 'P04S01'],
    ['7/30/2020 11:15:18 PM', '15', '4ML_10mm_2ND_STAGE', '15', 'None', '14', '14', 'P04S02'],
    ['7/30/2020 11:25:28 PM', '16', '4ML_10mm_2ND_STAGE', '16', 'None', '15', '15', 'P04S03'],
    ['7/30/2020 11:25:28 PM', '17', '4ML_10mm_2ND_STAGE', '17', 'None', '16', '16', 'P04S04']]


sample_names = ['M21535_5',
            'M21551_5',
            'M21523_5',
            'M24173_2',
            'M23131_6',
            'M23127_6',
            'L7547_3',
            'L40885_3',
            'L88759_2',
            'L88699_3',
            'L72331_3',
            'L72221_5',
            'L12023_6',
            'L73889_3']

In [65]:
date_fmt = '%m/%d/%Y %I:%M:%S %p'
date_fmt_2 = '%Y-%m-%d_%I-%M-%S-%p'
rename_tmpl = '{}_2_{}_(2)_{}_at_{}.csv'

In [66]:
def rename_uvfile_test(row, sample_name):
    dt = datetime.datetime.strptime(row[0],date_fmt) 
    dt_prefix = dt.strftime('%Y%m%d')
    dt_add = dt + datetime.timedelta(minutes=20)
    final_str = rename_tmpl.format(dt_prefix, sample_name, dt.strftime(date_fmt_2), dt_add.strftime('%I-%M-%S-%p'))
    return final_str

In [67]:
rename_uvfile_test(lst_rows[0], sample_names[0])

'20200730_2_M21535_5_(2)_2020-07-30_02-15-20-PM_at_02-35-20-PM.csv'

In [28]:
from random import randint
from shutil import copyfile

In [68]:
fp = '/Volumes/npsg/Gilson/RAW_UV_EXPORTS/'
csv_fi = [os.path.join(fp,fi) for fi in os.listdir(fp)]

In [69]:
csv_fi

['/Volumes/npsg/Gilson/RAW_UV_EXPORTS/07312020-163535_SAMPLETEST_(1)_2020-07-31_04-35-37-PM_at_04-51-23-PM.csv',
 '/Volumes/npsg/Gilson/RAW_UV_EXPORTS/1520030009_1520030010_secStg_273_276_G5_L11905_4_(50)_2020-03-20_11-04-48-AM_at_12-09-47-AM.csv',
 '/Volumes/npsg/Gilson/RAW_UV_EXPORTS/1520030009_1520030010_secStg_273_276_G5_L11905_4_(50)_2020-03-20_11-04-48-AM_at_12-09-47-AM_(2).csv',
 '/Volumes/npsg/Gilson/RAW_UV_EXPORTS/1520030009_1520030010_secStg_273_276_G5_L16485_6_(55)_2020-03-20_11-04-48-AM_at_01-30-43-AM.csv',
 '/Volumes/npsg/Gilson/RAW_UV_EXPORTS/1520030009_1520030010_secStg_273_276_G5_L16485_6_(55)_2020-03-20_11-04-48-AM_at_01-30-43-AM_(2).csv',
 '/Volumes/npsg/Gilson/RAW_UV_EXPORTS/1520030009_1520030010_secStg_273_276_G5_L18707_2_(52)_2020-03-20_11-04-48-AM_at_12-42-08-AM.csv',
 '/Volumes/npsg/Gilson/RAW_UV_EXPORTS/1520030009_1520030010_secStg_273_276_G5_L18707_2_(52)_2020-03-20_11-04-48-AM_at_12-42-08-AM_(2).csv',
 '/Volumes/npsg/Gilson/RAW_UV_EXPORTS/1520030009_1520030010

In [70]:
for i, sn in enumerate(sample_names): 
    copyfile(csv_fi[randint(0, len(csv_fi))-1], os.path.join(fp, rename_uvfile_test(lst_rows[i], sn)))

In [95]:
!ls /Volumes/npsg/Gilson/RAW_UV_EXPORTS/

07312020-163535_SAMPLETEST_(1)_2020-07-31_04-35-37-PM_at_04-51-23-PM.csv
1520030009_1520030010_secStg_273_276_G5_L11905_4_(50)_2020-03-20_11-04-48-AM_at_12-09-47-AM.csv
1520030009_1520030010_secStg_273_276_G5_L11905_4_(50)_2020-03-20_11-04-48-AM_at_12-09-47-AM_(2).csv
1520030009_1520030010_secStg_273_276_G5_L16485_6_(55)_2020-03-20_11-04-48-AM_at_01-30-43-AM.csv
1520030009_1520030010_secStg_273_276_G5_L16485_6_(55)_2020-03-20_11-04-48-AM_at_01-30-43-AM_(2).csv
1520030009_1520030010_secStg_273_276_G5_L18707_2_(52)_2020-03-20_11-04-48-AM_at_12-42-08-AM.csv
1520030009_1520030010_secStg_273_276_G5_L18707_2_(52)_2020-03-20_11-04-48-AM_at_12-42-08-AM_(2).csv
1520030009_1520030010_secStg_273_276_G5_L55521_3_(53)_2020-03-20_11-04-48-AM_at_12-58-19-AM.csv
1520030009_1520030010_secStg_273_276_G5_L55521_3_(53)_2020-03-20_11-04-48-AM_at_12-58-19-AM_(2).csv
1520030009_1520030010_secStg_273_276_G5_L55591_2_(54)_2020-03-20_11-04-48-AM_at_01-14-31-AM.csv
1520030009_1520030010_secStg_273_276_G5_L55591_

In [71]:
dt_str = '/usr/src/app/mnt/uvdata_files/20200730_2_M21535_5_(2)_07-30-2020_02-15-20-PM_at_02-35-20-PM.csv'
dt_str2 = '/usr/src/app/mnt/uvdata_files/1520030009_1520030010_secStg_273_276_G5_M22071_6_(57)_2020-03-20_11-04-48-AM_at_02-03-07-AM.csv'

In [93]:
REGEX_TIMESTAMP = r'(20)\d\d[-](0[1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])_\d{2}-\d{2}-\d{2}-[PA]M'
#REGEX_TIMESTAMP = r'(20)\d\d[- /.](0[1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])_\d{2}-\d{2}-\d{2}-[PA]M'

#datetime.datetime.strptime(re.search(REGEX_TIMESTAMP, dt_str).group(0), '%Y-%m-%d_%I-%M-%S-%p')


In [96]:
re.search(REGEX_TIMESTAMP, '20200730_2_L88699_3_(2)_2020-07-30_09-09-18-PM_at_09-29-18-PM.csv')

<re.Match object; span=(24, 46), match='2020-07-30_09-09-18-PM'>